In [27]:
from importlib import reload
import functions
reload(functions)
from functions import get_flight_ids, load_json_file,load_parquet_file,process_json_file, process_parquet_file, \
    remove_constant_columns, columns_to_scalar, get_common_columns, add_row_label, align_columns
import os
import pandas as pd
from joblib import dump, load
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime

# 1. Split Train-Test Data

In [2]:
data_dir = 'data' 
flight_ids = get_flight_ids(data_dir)
flight_ids.remove('StateDescriptions')
print(flight_ids)

['08412468-26ac-4777-9afb-4671f426277b', '0b3f3902-2c04-4625-8576-3bb963e3d709', '0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79', '1675a16e-a2a3-4038-9007-50b0b26a685c', '21194a58-8a4d-4d0f-a4a9-e374393183b4', '28bd3cd3-1d6a-403f-ab8a-83efaf260dd0', '2a2467dd-9cb2-45de-a64f-f182395b3d1a', '39b2c145-c49f-470b-8280-d253fa98153f', '663f573a-74c5-4368-b60b-1fb433cd835d', '8ac99efe-b70b-4b4b-983b-6064fe37c67b', '8c36586f-94e9-4ae9-8384-0f3342008677', '92b2d28b-21e4-498c-b6dd-c27a47716a25', '9a2e5b24-1d93-47ef-bd90-0fae0d719df7', 'a366ff0e-ac1e-4632-821f-594ee8750b90', 'a376807a-82d3-4526-b19f-98d4b3f9078b', 'b5a540db-434b-4c3d-86dd-4668d40586c2', 'd76bb0eb-bc08-4b35-8c1f-37369452083d', 'ef4852a4-fcfe-429b-b753-d11e2ad08cac', 'f40f71de-5cc2-4719-8a5a-abcf950cbd71']


In [26]:
test_ids = [
    "0b3f3902-2c04-4625-8576-3bb963e3d709",
    "663f573a-74c5-4368-b60b-1fb433cd835d",
    "8c36586f-94e9-4ae9-8384-0f3342008677",
    "a376807a-82d3-4526-b19f-98d4b3f9078b",
    "d76bb0eb-bc08-4b35-8c1f-37369452083d",
    "f40f71de-5cc2-4719-8a5a-abcf950cbd71"
]

#put the rest into train_ids first
train_ids = []
for i in flight_ids:
    if i not in test_ids:
        train_ids.append(i)

#split train_ids into train and validation ids

train_ids = [
    '08412468-26ac-4777-9afb-4671f426277b', 
    '0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79', 
    '1675a16e-a2a3-4038-9007-50b0b26a685c', 
    '21194a58-8a4d-4d0f-a4a9-e374393183b4', 
    '28bd3cd3-1d6a-403f-ab8a-83efaf260dd0', 
    '2a2467dd-9cb2-45de-a64f-f182395b3d1a', 
    '39b2c145-c49f-470b-8280-d253fa98153f', 
    '8ac99efe-b70b-4b4b-983b-6064fe37c67b', 
    '92b2d28b-21e4-498c-b6dd-c27a47716a25',
    '9a2e5b24-1d93-47ef-bd90-0fae0d719df7'
    ]

validate_ids=[
    'a366ff0e-ac1e-4632-821f-594ee8750b90',
    'b5a540db-434b-4c3d-86dd-4668d40586c2', 
    'ef4852a4-fcfe-429b-b753-d11e2ad08cac'
]

# 2. Train Data Process

## 2.1 Process and save the json files

In [ ]:
data_dir = 'data'
output_dir = 'train_data'
current_ids = train_ids

#output_dir = 'validate_data'
#current_ids = validate_ids

#output_dir = 'test_data'
#current_ids = test_ids


if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def save_json_data(flight_id):
    json_path = os.path.join(data_dir, f"{flight_id}.json")

    df = load_json_file(json_path)
    
    df = process_json_file(df)
    
    output_path = os.path.join(output_dir, f"{flight_id}.csv")
    df.to_csv(output_path, index=False)
    print(f"Saved {flight_id} data to {output_path}")

for flight_id in current_ids:
    save_json_data(flight_id)

Saved a366ff0e-ac1e-4632-821f-594ee8750b90 data to validate_data\a366ff0e-ac1e-4632-821f-594ee8750b90.csv
Saved b5a540db-434b-4c3d-86dd-4668d40586c2 data to validate_data\b5a540db-434b-4c3d-86dd-4668d40586c2.csv
Saved ef4852a4-fcfe-429b-b753-d11e2ad08cac data to validate_data\ef4852a4-fcfe-429b-b753-d11e2ad08cac.csv


In [51]:
# cache the processed json file
cache_dir = './cache/' + output_dir  
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in current_ids:
    file_path = os.path.join('./' + output_dir, f"{flight_id}.csv")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.csv', '_json.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_csv(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")


Cached data to ./cache/validate_data\a366ff0e-ac1e-4632-821f-594ee8750b90_json.joblib
Cached data to ./cache/validate_data\b5a540db-434b-4c3d-86dd-4668d40586c2_json.joblib
Cached data to ./cache/validate_data\ef4852a4-fcfe-429b-b753-d11e2ad08cac_json.joblib


## 2.2 Analyze StateDescriptions
Before we deal with the parquet files, first we have to analyze the StateDescription.json file.

In [7]:
import json
import pandas as pd

def analyze_state_descriptions(json_path):
    # Load JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Flatten JSON data and create a DataFrame
    records = []
    for name, entry in data.items():
        record = {
            'Name': name,
            'DataType': entry.get('dataType', 'Unknown'),
            'Unit': entry.get('unit', 'Unknown'),
            'StateID': entry.get('stateId', 'Unknown'),
            'Persistence': entry.get('isPersistent', False),
        }
        
        # Add default values for different models
        for model, default_value in entry.get('DefaultValues', {}).items():
            record[f'Default_{model}'] = default_value

        # Add display units for different models
        for model, display_unit in entry.get('DefaultDisplayUnits', {}).items():
            record[f'Unit_{model}'] = display_unit

        records.append(record)
    
    df = pd.DataFrame(records)
    
    # Display general information about the DataFrame
    print("General Info:")
    print(df.info())
    
    # Display summary statistics for each column
    print("\nSummary Statistics:")
    print(df.describe(include='all'))
    
    # Count unique data types and their frequency
    print("\nData Type Counts:")
    print(df['DataType'].value_counts())
    
    # Count unique units and their frequency
    print("\nUnit Counts:")
    print(df['Unit'].value_counts())
    
    # Check persistence flag distribution
    print("\nPersistence Flag Distribution:")
    print(df['Persistence'].value_counts())
    
    return df


In [8]:

json_path = './data/StateDescriptions.json'
df_states = analyze_state_descriptions(json_path)

# save to file
output_path = './state_descriptions_analysis.csv'
df_states.to_csv(output_path, index=False)
print(f"\nData saved to {output_path}")

# Display the first few rows for inspection
print("\nFirst few rows of the data:")
print(df_states.head())

General Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         1746 non-null   object
 1   DataType     1746 non-null   object
 2   Unit         381 non-null    object
 3   StateID      1746 non-null   int64 
 4   Persistence  1746 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 56.4+ KB
None

Summary Statistics:
                      Name DataType Unit       StateID Persistence
count                 1746     1746  381  1.746000e+03        1746
unique                1746        8   22           NaN           2
top     VideoStream_Active   Double  0-1           NaN       False
freq                     1     1162   99           NaN        1424
mean                   NaN      NaN  NaN  2.118100e+09         NaN
std                    NaN      NaN  NaN  1.232422e+09         NaN
min                    NaN      NaN  N

## 2.3 Process and save parquet data

### 2.3.1 Cache parquet raw data

In [10]:
cache_dir = './cache/raw'  
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in flight_ids:
    file_path = os.path.join('./data', f"{flight_id}.parquet")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.parquet', '.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_parquet(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")

Cached data to ./cache/raw\08412468-26ac-4777-9afb-4671f426277b.joblib
Cached data to ./cache/raw\0b3f3902-2c04-4625-8576-3bb963e3d709.joblib
Cached data to ./cache/raw\0bbf0c4e-fb3c-4213-bff8-ef21ee5ebf79.joblib
Cached data to ./cache/raw\1675a16e-a2a3-4038-9007-50b0b26a685c.joblib
Cached data to ./cache/raw\21194a58-8a4d-4d0f-a4a9-e374393183b4.joblib
Cached data to ./cache/raw\28bd3cd3-1d6a-403f-ab8a-83efaf260dd0.joblib
Cached data to ./cache/raw\2a2467dd-9cb2-45de-a64f-f182395b3d1a.joblib
Cached data to ./cache/raw\39b2c145-c49f-470b-8280-d253fa98153f.joblib
Cached data to ./cache/raw\663f573a-74c5-4368-b60b-1fb433cd835d.joblib
Cached data to ./cache/raw\8ac99efe-b70b-4b4b-983b-6064fe37c67b.joblib
Cached data to ./cache/raw\8c36586f-94e9-4ae9-8384-0f3342008677.joblib
Cached data to ./cache/raw\92b2d28b-21e4-498c-b6dd-c27a47716a25.joblib
Cached data to ./cache/raw\9a2e5b24-1d93-47ef-bd90-0fae0d719df7.joblib
Cached data to ./cache/raw\a366ff0e-ac1e-4632-821f-594ee8750b90.joblib
Cached

### 2.3.2 process parquet data and cache

##### 2.3.2.1 See the structures to adjust the function "process_parquet_file"

In [11]:

# see the original structure os the parquet file
data = pd.read_parquet('./data/2a2467dd-9cb2-45de-a64f-f182395b3d1a.parquet')
print(data[3501046967])

TimeStamp                         FrameCounter
2024-02-14 06:26:55.328843+00:00  0               0.0
2024-02-14 06:26:55.338647+00:00  1               NaN
2024-02-14 06:26:57.816582+00:00  2               NaN
2024-02-14 06:26:57.831172+00:00  3               NaN
2024-02-14 06:26:57.859734+00:00  4               NaN
                                                 ... 
2024-02-14 06:45:40.456558+00:00  289727          NaN
2024-02-14 06:45:40.469469+00:00  289728          NaN
2024-02-14 06:45:40.488957+00:00  289729          NaN
2024-02-14 06:45:40.502970+00:00  289730          NaN
2024-02-14 06:45:40.510829+00:00  289731          NaN
Name: 3501046967, Length: 205635, dtype: float64


In [ ]:
# See the structure of the parquet file after process

df = load_parquet_file('./data/08412468-26ac-4777-9afb-4671f426277b.parquet')
df = process_parquet_file(df)
print(df.head())

In [13]:
# See the structure of the parquet file after remove unnecessay columns

df= remove_constant_columns(df)

In [14]:
print(df.head()) 

                                               117182271   156875280   \
TimeStamp                        FrameCounter                           
2024-02-14 06:46:58.397576+00:00 0               2.312171       False   
2024-02-14 06:46:58.416348+00:00 1               2.312171       False   
2024-02-14 06:47:00.886922+00:00 2               2.309550       False   
2024-02-14 06:47:00.894921+00:00 3               2.309550       False   
2024-02-14 06:47:00.926924+00:00 4               2.309550       False   

                                                353872548   381802901   \
TimeStamp                        FrameCounter                            
2024-02-14 06:46:58.397576+00:00 0             135500000.0         1.0   
2024-02-14 06:46:58.416348+00:00 1             135500000.0         1.0   
2024-02-14 06:47:00.886922+00:00 2             135500000.0         1.0   
2024-02-14 06:47:00.894921+00:00 3             135500000.0         1.0   
2024-02-14 06:47:00.926924+00:00 4          

##### 2.3.2.2 (Optional) Here there is some mistake that the files are saved to .csv, but remaining the same with .parquet is better. 
Since reprocessing it took a lot of time, so I just wrote the next part to transfer from csv to parquet file.
If you haven't processed the data, skip this part and next (Convert "_parquet.csv" to ".parquet"), just run the part of " Load from cache and process parquet data"

In [ ]:
data_dir = 'data'
output_dir = 'test_data'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to process and save Parquet file data
def save_parquet_data(flight_id):
    parquet_path = os.path.join(data_dir, f"{flight_id}.parquet")

    df = load_parquet_file(parquet_path)
     
    df = process_parquet_file(df)

    df = columns_to_scalar(df) # [] -> float | string -> NaN | bool -> int | ...

    df = remove_constant_columns(df)
    
    # Define the base output file name
    output_path = os.path.join(output_dir, f"{flight_id}_parquet.csv")
    
    # Save the DataFrame as a CSV file
    df.to_csv(output_path, index=False)
    print(f"Saved {flight_id} Parquet data to {output_path}")

# Process each Parquet file and save it
for flight_id in train_ids: 
    # check if the file exists
    output_path = os.path.join(output_dir, f"{flight_id}_parquet.csv")
    if os.path.exists(output_path):
        print(f"File {output_path} already exists. Skipping save.")
    else:
        save_parquet_data(flight_id)

##### 2.3.2.3 (Optional) Convert "_parquet.csv" to ".parquet"
If you need to process the parquet data, skip this part. Run next part

In [32]:

input_dir = "./train_data" 
output_dir = "./train_data" 

os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.endswith("_parquet.csv"): 
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name.replace("_parquet.csv", ".parquet"))
        
        try:
            df = pd.read_csv(input_path)
            df.to_parquet(output_path, index=False)
            print(f"Converted {file_name} to {output_path}")
            
            os.remove(input_path)
            print(f"Deleted original file: {input_path}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

##### 2.3.2.4 Load from cache and process parquet data

In [29]:
data_dir = './cache/raw'

configurations = [
    ('test_data', test_ids),
    ('train_data', train_ids),
    ('validate_data', validate_ids)
]

# after running get_common_columns (took nearly 2h) we can store the common columns to speed up the process
# common columns: all columns which are at least in one flight not fully constant
common_columns = ['6417134', '28827303', '45268905', '108790465', '117182271', '156751662', '156875280', '192589766', '203733126', '301190512', '340940040', '353872548', '362955152', '381802901', '419755350', '439066970', '491152707', '513685691', '518160218', '557098463', '564468419', '580739016', '610037603', '614270119', '630411227', '637219977', '638103778', '677695791', '683661709', '683679735', '705547489', '731865654', '738096043', '750466567', '754202368', '760531114', '786775081', '853651284', '855504072', '919467553', '943710121', '965636185', '1049384201', '1049691088', '1074440103', '1089322549', '1102716554', '1105598667', '1112206407', '1114842726', '1128128022', '1129907732', '1145477697', '1169465987', '1185350318', '1269872157', '1289350632', '1296641768', '1308053202', '1314928026', '1324901455', '1359970420', '1432743115', '1450782446', '1464341275', '1464691036', '1500331376', '1501357761', '1515452476', '1550781114', '1553784294', '1556708778', '1637431807', '1666704728', '1748585914', '1782666709', '1791424934', '1796138214', '1814846759', '1831637452', '1847367549', '1863207721', '1879505128', '1895873384', '1909389254', '1971234014', '1984892607', '2074986628', '2122819504', '2142867155', '2155694236', '2176349777', '2209591699', '2219783417', '2275361864', '2279678027', '2298125514', '2318747298', '2338387453', '2356154278', '2370500633', '2381936850', '2400822283', '2457184866', '2478351513', '2496586604', '2498379844', '2503582969', '2524343919', '2531864991', '2545498036', '2546226082', '2589702191', '2590494337', '2663217327', '2690567179', '2700477795', '2705846059', '2716830861', '2730101906', '2769157819', '2815095005', '2829673346', '2840355407', '2876656444', '2885619987', '2936255604', '2942017749', '2979223644', '2988014268', '3063191758', '3066527403', '3163423083', '3207956524', '3296928681', '3298183894', '3298276793', '3321734281', '3327788219', '3335916359', '3355805873', '3358452658', '3363211662', '3410338846', '3489008753', '3493828033', '3495907791', '3502630583', '3576014097', '3590140009', '3599019297', '3599416138', '3645293884', '3659998070', '3663402178', '3687766012', '3706315026', '3720232134', '3742756340', '3752743700', '3771464240', '3827725929', '3871827618', '3881043328', '3911364189', '3917330120', '3928845979', '3933014103', '3966668421', '3981973171', '4015812076', '4021288050', '4022486067', '4054750327', '4056189073', '4058842283', '4061964072', '4065458540', '4066202059', '4081583215', '4096114806', '4096188490', '4174711423', '4210968644', '4211937095', '4233706951', '4247030081', '4264003232', '4294814812']
#common_columns = get_common_columns(data_dir, configurations)
# after running the model and using lasso, we identified the necessarry columns: (when model/data changes, this could change)
#common_columns = []

for folder, ids in configurations:
    # Create the output directory if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Function to process and save Parquet file data
    def save_parquet_data(flight_id):
        print(f"{datetime.now()}: Processing flight {flight_id}")
        parquet_path = os.path.join(data_dir, f"{flight_id}.joblib")

        df = load(parquet_path)
        df.columns = df.columns.map(str)
        
        # keep only the common columns in each DataFrame
        df = df.reindex(columns=common_columns, fill_value=0)
        
        df = process_parquet_file(df)

        df = columns_to_scalar(df) # [] -> float | string -> NaN | bool -> int | ...
    
        df = df.reindex(columns=common_columns, fill_value=0)

        # add labels according to json
        df = add_row_label(df, folder, flight_id)

        # Define the base output file name
        output_path = os.path.join(folder, f"{flight_id}.parquet")

        # Save the DataFrame as a CSV file
        df.columns = df.columns.map(str)
        df.to_parquet(output_path, index=True)
        print(f"{datetime.now()}: Saved {flight_id} Parquet data to {output_path}")

    # Process each Parquet file and save it
    for flight_id in ids:
        # check if the file exists
        output_path = os.path.join(folder, f"{flight_id}.parquet")
        if os.path.exists(output_path):
            print(f"File {output_path} already exists. Skipping save.")
        else:
            save_parquet_data(flight_id)

File test_data\0b3f3902-2c04-4625-8576-3bb963e3d709.parquet already exists. Skipping save.
2024-11-24 04:43:04.796253: Processing flight 663f573a-74c5-4368-b60b-1fb433cd835d
2024-11-24 04:47:34.462837: Saved 663f573a-74c5-4368-b60b-1fb433cd835d Parquet data to test_data\663f573a-74c5-4368-b60b-1fb433cd835d.parquet
2024-11-24 04:47:34.535458: Processing flight 8c36586f-94e9-4ae9-8384-0f3342008677
2024-11-24 04:53:08.580219: Saved 8c36586f-94e9-4ae9-8384-0f3342008677 Parquet data to test_data\8c36586f-94e9-4ae9-8384-0f3342008677.parquet
2024-11-24 04:53:08.624396: Processing flight a376807a-82d3-4526-b19f-98d4b3f9078b
2024-11-24 04:57:56.993207: Saved a376807a-82d3-4526-b19f-98d4b3f9078b Parquet data to test_data\a376807a-82d3-4526-b19f-98d4b3f9078b.parquet
File test_data\d76bb0eb-bc08-4b35-8c1f-37369452083d.parquet already exists. Skipping save.
File test_data\f40f71de-5cc2-4719-8a5a-abcf950cbd71.parquet already exists. Skipping save.
File train_data\08412468-26ac-4777-9afb-4671f426277b

### 2.3.3 Cache parquet processed data

In [30]:
#output_dir = 'test_data'
#current_ids = test_ids

#output_dir = 'train_data'
#current_ids = train_ids

output_dir = 'validate_data'
current_ids = validate_ids

cache_dir = './cache/' + output_dir
os.makedirs(cache_dir, exist_ok=True) 

# cache parquet_csv file
for flight_id in current_ids:
    file_path = os.path.join('./' + output_dir, f"{flight_id}.parquet")

    cache_file = os.path.join(cache_dir, os.path.basename(file_path).replace('.parquet', '_parquet.joblib'))
    
    if os.path.exists(cache_file):
        print(f"{cache_file} cached already")
    else:
        data = pd.read_parquet(file_path)
        dump(data, cache_file)
        print(f"Cached data to {cache_file}")

./cache/validate_data\a366ff0e-ac1e-4632-821f-594ee8750b90_parquet.joblib cached already
Cached data to ./cache/validate_data\b5a540db-434b-4c3d-86dd-4668d40586c2_parquet.joblib
Cached data to ./cache/validate_data\ef4852a4-fcfe-429b-b753-d11e2ad08cac_parquet.joblib


# 3. Visualize the data